In [8]:
import numpy as np
import pandas as pd
from scipy.signal import correlate
import matplotlib.pyplot as plt
from scipy.sparse import lil_matrix
from itertools import product
from itertools import combinations
from datetime import datetime, timedelta
from multiprocessing import Pool


# Load the temporal series data
data_with_date = pd.read_csv('temporal_series_10.csv')
data = data_with_date.drop(columns=['date'])

#Inizialize parameters
max_tau = 15 
threshold = 0.05

#Function to compute maximum of time correlation function between two temporal series
def time_delayed_cross_correlation(df, col1, col2, max_tau, threshold=0.05):
    """
    Optimized custom time-delayed cross-correlation function for two time series.

    Args:
    df (pandas.DataFrame): Dataframe containing the time series data.
    col1 (str): Name of the first time series column.
    col2 (str): Name of the second time series column.
    max_tau (int): Maximum time delay (positive or negative).
    threshold (float): Threshold for maximum correlation.

    Returns:
    float: Maximum cross-correlation above the threshold, or 0 if none.
    """
    # Extract the series and compute means and std deviations
    series_i = df[col1].values
    series_j = df[col2].values
    mean_i, std_i = series_i.mean(), series_i.std()
    mean_j, std_j = series_j.mean(), series_j.std()
    
    # Check if standard deviations are zero to avoid division by zero
    if std_i == 0 or std_j == 0:
        return 0
    
    # Initialize array to store cross-correlations for each lag
    cross_corrs = np.zeros(2 * max_tau + 1)

    # Compute cross-correlation for each lag
    for tau in range(-max_tau, max_tau + 1):
        if tau < 0:
            # Positive lag: shift series_j forward by -tau (series_i aligns with delayed series_j)
            numerator = np.sum((series_i[-tau:] - mean_i) * (series_j[:len(series_j) + tau] - mean_j))
            denominator = (std_i * std_j * (len(series_i) + tau))
        else:
            # Negative or zero lag: shift series_i forward by tau (series_j aligns with delayed series_i)
            numerator = np.sum((series_i[:len(series_i) - tau] - mean_i) * (series_j[tau:] - mean_j))
            denominator = (std_i * std_j * (len(series_i) - tau))

        # Calculate cross-correlation, check for zero denominator
        cross_corrs[tau + max_tau] = numerator / denominator if denominator != 0 else 0

    # Find the maximum correlation and compare with threshold
    max_corr = np.max(cross_corrs)
    max_index = np.argmax(cross_corrs)
    best_tau = max_index - max_tau  # Convert array index back to tau value

    # Apply threshold check and adjust max_corr based on tau direction
    if max_corr >= threshold:
        return max_corr if best_tau >= 0 else -max_corr
    else:
        return 0


#Function to compute the whole adjacency matrix (impossible to compute)
def ccr_matrix(df):
    #build the adjacency matrix
    adjacency_matrix = pd.DataFrame(np.nan, index=df.columns, columns=df.columns)

    #iterate for each pair of columns
    for col1 in df.columns:
        for col2 in df.columns:
            adjacency_matrix.loc[col1,col2] = time_delayed_cross_correlation(df, col1, col2, max_tau)

    return adjacency_matrix

#function to compute just a submatrix of the adjacency matrix (submatricese on the diagonal)
def compute_submatrix(df, columns, filename, max_tau):
    submatrix = pd.DataFrame(index=columns, columns=columns)

    # Loop through column pairs, skipping redundant calculations
    for col1, col2 in product(columns, repeat=2):
        if col1 <= col2:  # Ensures each pair is calculated only once
            correlation = time_delayed_cross_correlation(df, col1, col2, max_tau)
            submatrix.loc[col1, col2] = correlation
            submatrix.loc[col2, col1] = correlation  # Fill symmetric position

    submatrix.to_csv(filename)
    print(f"Submatrix {filename} has been saved.")
    return submatrix

#function to computer a submatrix which is not on the diagonal
def compute_cross_group_matrix(df, group_a, group_b, filename):
    cross_matrix = pd.DataFrame(index=group_a, columns=group_b)
    for col1, col2 in product(group_a, group_b):
        cross_matrix.loc[col1, col2] = time_delayed_cross_correlation(df, col1, col2, max_tau)
    cross_matrix.to_csv(filename)
    print(f"submatrix{filename} has been saved")
    return cross_matrix

#Now to compute the submatrices we need to divide the 3104 columns into groups
def divide_into_groups(df, group_sizes):
    #Divides the comlumns into specified sizes
    columns = df.columns.tolist()
    groups = []
    start = 0
    for size in group_sizes:
        groups.append(columns[start:start + size])
        start += size
    return groups

#Change of program, now we will try to compute manually the single pieces
group_1 = data.columns[0:1000]
group_2 = data.columns[1000:2000]
group_3 = data.columns[2000:3104]


filename_sub = "10_sub_mat_3.csv"
compute_submatrix(data, group_3, filename_sub, max_tau)

#filename_cross = "9_cross_matrix_2_3.csv"
#compute_cross_group_matrix(data, group_2, group_3, filename_cross)


Submatrix 10_sub_mat_2.csv has been saved.


,21083.0,21085.0,21087.0,21089.0,21091.0,21093.0,21095.0,21097.0,21099.0,21101.0,...,38079.0,38081.0,38083.0,38085.0,38087.0,38089.0,38091.0,38093.0,38095.0,38097.0
21083.0,1.0,0.662843,0.537601,0.759518,0.747386,0.768865,0.766321,0.770229,0.537485,0.689247,...,0.591795,-0.11767,0,-0.478702,-0.401702,-0.467957,-0.36351,-0.62183,-0.542804,0.442131
21085.0,0.662843,1.0,0.503523,0.573767,0.670286,0.708532,0.609639,0.589042,0.482617,0.679851,...,-0.514497,-0.096443,-0.124327,-0.48895,-0.328028,-0.396525,-0.280783,-0.591393,-0.4974,-0.367106
21087.0,0.537601,0.503523,1.0,-0.397032,0.548965,0.54883,0.431524,0.468039,-0.410625,-0.468865,...,0.374303,-0.187092,0.175291,-0.366123,-0.261783,0.282587,0.231286,0.39851,0.352045,0.375486
21089.0,0.759518,0.573767,-0.397032,1.0,0.607819,0.700421,0.695395,0.582652,0.413873,0.670545,...,0.607734,-0.122804,0.074482,0.494592,-0.444612,-0.410668,-0.482715,-0.50586,-0.462668,-0.374674
21091.0,0.747386,0.670286,0.548965,0.607819,1.0,0.839301,0.680363,0.671446,0.531774,0.684658,...,-0.523871,-0.123849,-0.071085,-0.46788,-0.400177,-0.4154,-0.380878,-0.712604,-0.426502,-0.341843
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38089.0,-0.467957,-0.396525,0.282587,-0.410668,-0.4154,-0.465148,-0.448238,-0.445883,-0.171642,-0.362173,...,0.483708,0.208626,0.215068,-0.432374,-0.289435,1.0,-0.301246,-0.450261,-0.397465,0.27676
38091.0,-0.36351,-0.280783,0.231286,-0.482715,-0.380878,-0.373154,-0.304361,-0.426132,0.137884,-0.300329,...,0.357122,0.123393,-0.150254,-0.291782,-0.289471,-0.301246,1.0,0.384609,-0.38815,-0.239665
38093.0,-0.62183,-0.591393,0.39851,-0.50586,-0.712604,-0.655118,-0.512513,-0.672606,-0.41306,-0.565807,...,-0.629171,0.257669,0.169211,-0.430599,0.373638,-0.450261,0.384609,1.0,0.465631,0.419561
38095.0,-0.542804,-0.4974,0.352045,-0.462668,-0.426502,-0.448448,-0.49871,-0.482605,-0.333766,-0.422826,...,0.453897,0.175604,0.135733,0.538717,-0.437271,-0.397465,-0.38815,0.465631,1.0,0.493906
